# Fine Tune GPT-модели с LoRA
В этом ноутбуке проводится дообучение GPT-like модели `ai-forever/rugpt3medium_based_on_gpt2` с использованием метода LoRA, для ускорения обучения и снижения тебования к памяти.

Цель — научить модель генерировать маркетинговые описания одежды по названию и характеристикам.

Решаю задачу как Causal LM. Модель учится предсказывать следующий токен по предыдущим. В качестве входа подаются характеристики товара, а модель учится дописывать за ними связное описание.

In [ ]:
!pip install U transformers peft datasets accelerate bitsandbytes sentencepiece evaluate nltk rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.5 MB/s eta 0:00:00
  

## 1. Загрузка и подготовка данных

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os
import pandas as pd
import numpy as np
from datasets import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# беру те же датасеты, что и в rut5
df_train = pd.read_csv('drive/MyDrive/proj_cloth_desc_gen/df_train.csv', index_col =0).reset_index(drop=True)
df_val = pd.read_csv('drive/MyDrive/proj_cloth_desc_gen/df_val.csv', index_col =0).reset_index(drop=True)
df_test = pd.read_csv('drive/MyDrive/proj_cloth_desc_gen/df_test.csv', index_col =0).reset_index(drop=True)

# удаляю инструкцию
df_train["input"] = df_train["input"].str.replace("Сгенерируй описание одежды для карточки товара:\n", "", regex=False)
df_val["input"] = df_val["input"].str.replace("Сгенерируй описание одежды для карточки товара:\n", "", regex=False)
df_test["input"] = df_test["input"].str.replace("Сгенерируй описание одежды для карточки товара:\n", "", regex=False)

df_train.shape, df_val.shape, df_test.shape

((44727, 7), (4970, 7), (21299, 7))

## 2. Преобразование данных для causal LM и токенизация
В своей задаче использую русскоязычную GPT-2 `ai-forever/rugpt3medium_based_on_gpt2` от Sber AI

In [ ]:
# подготовка данных для подачи в модель
system = (
    "Ты генерируешь правдивые описания одежды только по данным характеристикам. "
    "Не выдумывай состав, цвета, бренды и параметры, которых нет во входе."
)

def format_example(row):
    return (
        f"<s>### СИСТЕМА:\n{system}\n\n"
        f"### ХАРАКТЕРИСТИКИ:\n{row['input']}\n\n"
        f"### ОПИСАНИЕ:\n{row['final_desc']}\n</s>"
    )

df_train['text'] = df_train.apply(format_example, axis=1)
df_val['text'] = df_val.apply(format_example, axis=1)

In [ ]:
df_train["input_len"] = df_train["text"].apply(lambda x: len(tokenizer(x)["input_ids"]))
df_train["input_len"].describe()

,input_len
count,44727.000000
mean,401.877613
std,119.446750
min,121.000000
25%,314.000000
50%,392.000000
75%,489.000000
max,1381.000000


In [ ]:
df_train["output_len"] = df_train["final_desc"].apply(lambda x: len(tokenizer(x)["input_ids"]))
df_train["output_len"].describe()

,output_len
count,44727.000000
mean,270.043039
std,110.174308
min,42.000000
25%,188.000000
50%,258.000000
75%,353.000000
max,1188.000000


In [ ]:
train_ds = Dataset.from_pandas(df_train[['text']])
val_ds   = Dataset.from_pandas(df_val[['text']])

In [ ]:
# загрузка токенизатора
from transformers import AutoTokenizer

model_name = "ai-forever/rugpt3medium_based_on_gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [ ]:
# токенизация
max_len = 768

def tokenize(batch):
    out = tokenizer(
        batch["text"],
        truncation=True,
        max_length=max_len,
        padding="max_length",
        return_tensors=None,
    )
    # Для Сausal LM вход и выход совпадают labels = input_ids (сдвиг внутри модели)
    out["labels"] = out["input_ids"].copy()
    return out

tokenized_train = train_ds.map(tokenize, batched=True, remove_columns=["text"])
tokenized_val   = val_ds.map(tokenize,   batched=True, remove_columns=["text"])

Map:   0%|          | 0/44727 [00:00<?, ? examples/s]

Map:   0%|          | 0/4970 [00:00<?, ? examples/s]

## 3. Fine-tune GPT-like модели
- Обучаю модель на GPU A100 с использованием fp16 для ускорения.
- Для fine-tuning применяю LoRA: подключаю адаптеры к слоям внимания и проекций, обучаю только их, а базовые веса модели остаются замороженными.
- Лучшая модель выбирается автоматически по eval_loss. Для causal LM это наиболее корректный способ выбора, так как модель оптимизируется именно на минимизацию функции потерь при предсказании следующего токена.
- В отличие от seq2seq-подхода (T5), здесь не используется генерация на этапе обучения: модель оценивается напрямую по лоссу.
- Также применяю раннюю остановку, чтобы прерывать обучение, если качество перестаёт улучшаться, и избежать переобучения.


In [ ]:
import torch
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

In [ ]:
# загрузка базовой модели
model = AutoModelForCausalLM.from_pretrained(model_name)

# конфигурация LoRA
# добавляем LoRA только в слои внимания и проекций
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn","c_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# оборачиваю модель в PEFT (LoRA)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:2156: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 4,325,376 || all params: 360,197,120 || trainable%: 1.2008


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
!nvidia-smi

Sat Aug  9 17:23:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   36C    P8             16W /   72W |       3MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import gc

torch.cuda.empty_cache()
gc.collect()

72

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback

out_dir = "/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results"


training_args = TrainingArguments(
    output_dir=out_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,    
    eval_accumulation_steps=32,            
    num_train_epochs=5,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",    
    greater_is_better=False,             
    logging_strategy="steps",
    logging_steps=100,
    logging_first_step=True,
    learning_rate=2e-4,
    warmup_steps=100,
    lr_scheduler_type="cosine",
    fp16=True,
    fp16_full_eval=True,
    report_to="none",                   
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


In [ ]:
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/checkpoint-5500")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
6000,1.578000,1.515112
6500,1.563400,1.515599


TrainOutput(global_step=6989, training_loss=0.334756307292179, metrics={'train_runtime': 7094.895, 'train_samples_per_second': 31.521, 'train_steps_per_second': 0.985, 'total_flos': 3.15992313298944e+17, 'train_loss': 0.334756307292179, 'epoch': 5.0})

In [ ]:
# сохраняем LoRA-адаптеры
trainer.save_model(out_dir)   
tokenizer.save_pretrained(out_dir)

('/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/tokenizer_config.json',
 '/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/special_tokens_map.json',
 '/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/vocab.json',
 '/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/merges.txt',
 '/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/added_tokens.json',
 '/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/tokenizer.json')

## 4. Генерация текста
Беру сэмпл из 200 примеров из тестовой выборки. Для каждого примера генерирую описание и сохраняю результат в CSV. 

Используемые параметры генерации:
- num_beams=5, do_sample=False - beam search для уменьшения вариативности и галлюцинаций
- max_new_tokens=300 - ограничение длины (средняя длина таргета ≈270 токенов)
- no_repeat_ngram_size=4, repetition_penalty=1.25 - защита от повторов и клише
- length_penalty=0.9 - лёгкий штраф по длине для компактных описаний

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import json, os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# слияние LoRA-адаптера с базовой моделью и сохранение итоговой версии

# базовая модель
base_model = "ai-forever/rugpt3medium_based_on_gpt2"
# дообученный LoRA-адаптер
adapter_dir = "/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results"
# сохраняем сюда итоговую модель
output_dir = "/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/merged_model"

# загрузка базовой моделт
base = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype="auto", device_map="auto")
# подгружаем в базовую модель LoRA-адаптер
model = PeftModel.from_pretrained(base, adapter_dir)
# слияние весов адаптера и базовой модели
merged = model.merge_and_unload()
# сохранение итоговой модели
merged.save_pretrained(output_dir, safe_serialization=True)

# загружаем и сохраняем токенизатор в ту же папку где хранится итоговая модель
tokenizer = AutoTokenizer.from_pretrained(adapter_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/merged_model/tokenizer_config.json',
 '/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/merged_model/special_tokens_map.json',
 '/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/merged_model/vocab.json',
 '/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/merged_model/merges.txt',
 '/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/merged_model/added_tokens.json',
 '/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/merged_model/tokenizer.json')

In [ ]:
# загрузка финальной модели
model_dir = "/content/drive/MyDrive/proj_cloth_desc_gen/gpt_lora_results/merged_model"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_dir, torch_dtype=torch.float16, device_map="auto")
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
df_test = pd.read_csv('drive/MyDrive/proj_cloth_desc_gen/df_test.csv', index_col =0).reset_index(drop=True)
df_test['input'] = df_test['input'].str.replace('Сгенерируй описание одежды для карточки товара:\n','')
df_test.head()

In [ ]:
import re

system = (
    "Ты генерируешь правдивые описания одежды только по данным характеристикам. "
    "Не выдумывай состав, цвета, бренды и параметры, которых нет во входе."
)

# функция подготовки промпта
def build_prompt(input_text: str):
    return (
        f"### СИСТЕМА:\n{system}\n\n"
        f"### ХАРАКТЕРИСТИКИ:\n{input_text.strip()}\n\n"
        f"### ОПИСАНИЕ:\n"
    )


# функция генерации 
@torch.no_grad()
def generate_description(model, tokenizer, input_text, gen_params, stop_regex=None):
    # Преобразование входного текста для подачи в модель
    prompt = build_prompt(input_text)

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.eos_token_id
        
    # токенизация входа
    enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    # генерация
    with torch.no_grad():
        out = model.generate(**enc, **gen_params)
        
    # декодирование в текст
    generated_text = tokenizer.decode(out[0], skip_special_tokens=True)

    # если модель повторила промпт - убираем его, оставляя только сам текст описания
    if "### ОПИСАНИЕ:" in generated_text:
        generated_text = generated_text.split("### ОПИСАНИЕ:")[-1].strip()

    # обрезаем по последнему знаку конца предложения: .,!,?, чтобы не было обрывков
    cut = max(generated_text.rfind("."), generated_text.rfind("!"), generated_text.rfind("?"))
    if cut != -1:
        generated_text = generated_text[:cut+1]

    # по желанию можно удалить другие ненужные фрагменты
    if stop_regex:
        generated_text = re.sub(stop_regex, "", generated_text, flags=re.IGNORECASE).strip()
    return generated_text


In [ ]:
df_sample = df_test.sample(n=3)
inputs = df_sample.input.tolist()
outputs = df_sample.final_desc.tolist()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
gen_params = dict(
    num_beams=5,                
    do_sample=False,            
    max_new_tokens=300,        
    no_repeat_ngram_size=4,     
    repetition_penalty=1.25,   
    length_penalty=0.9,         
    early_stopping=True
)

for in_, out_ in zip(inputs, outputs):
  print('--'*50)
  print(f"Input text:\n{in_}\n")
  gen = generate_description(model, tokenizer, in_, gen_params)
  print(f"Generated text:\n{gen}\n")
  print()


----------------------------------------------------------------------------------------------------
Input text:
Наименование товара: Джинсы трубы широкие
Утеплитель: без утеплителя
Тип посадки: высокая посадка
Страна производства: Россия
Вид застежки: молния; пуговицы
Особенности модели: прямые
Декоративные элементы: широкие
Тип карманов: прорезные
Комплектация: джинсы клеш - 1 шт

Generated text:
Джинсы трубы широкие от бренда это идеальныи выбор для мужчин, ищущих комфорт и стиль в повседневнои одежде. Эти джинсы изготовлены из высококачественного материала, состоящего на 80% из хлопка и на 20% из полиэстера, что обеспечивает не только удобство ношения, но и долговечность изделия.

Особенностью этих джинсов является их универсальность: они подходят как для повседневного ношения, так и для особых случаев, таких как свадьба, корпоратив или выпускнои. Благодаря своему классическому дизаину, они легко сочетаются с любои одеждои, создавая модныи и стильныи образ.

Выбирая джинсы трубы ши

In [ ]:
gen_params = dict(
    num_beams=3,
    do_sample=True,
    temperature=0.6,         
    top_p=0.9,
    max_new_tokens=300,
    no_repeat_ngram_size=4,
    repetition_penalty=1.2,
    length_penalty=0.95,
    early_stopping=True
)

for in_, out_ in zip(inputs, outputs):
  print('--'*50)
  print(f"Input text:\n{in_}\n")
  gen = generate_description(model, tokenizer, in_, gen_params)
  print(f"Generated text:\n{gen}\n")
  print()

----------------------------------------------------------------------------------------------------
Input text:
Наименование товара: Джинсы трубы широкие
Утеплитель: без утеплителя
Тип посадки: высокая посадка
Страна производства: Россия
Вид застежки: молния; пуговицы
Особенности модели: прямые
Декоративные элементы: широкие
Тип карманов: прорезные
Комплектация: джинсы клеш - 1 шт

Generated text:
Джинсы трубы широкие от бренда это идеальныи выбор для мужчин, ищущих комфорт и стиль в повседневнои одежде. Эти джинсы изготовлены из высококачественного денима, состоящего на 70% из хлопка и на 30% из полиэстера, что обеспечивает не только мягкость, но и долговечность изделия.

Особенностью этих джинсов является их универсальность: они подходят как для повседневного ношения, так и для особых случаев, таких как свадьба, выпускнои или корпоратив. Джинсы клеш - это классика, которая никогда не выидет из моды.

Эти джинсы имеют широкии крои, что делает их идеальным выбором как для высоких, так

In [ ]:
sample_df = pd.read_csv('drive/MyDrive/proj_cloth_desc_gen/df_sample_pred.csv', index_col = 0)
sample_df.head()

In [ ]:
gen_params = dict(
    num_beams=5,               
    do_sample=False,
    max_new_tokens=300,         
    no_repeat_ngram_size=4,
    repetition_penalty=1.25,    
    length_penalty=0.9,         
    early_stopping=True
)


sample_df["gpt_gen"] = sample_df["input"].apply(lambda x: generate_description(model, tokenizer, x, gen_params))

In [ ]:
sample_df.to_csv('drive/MyDrive/proj_cloth_desc_gen/df_sample_pred.csv')